In [1]:
import cv2 as cv
import numpy as np
import pandas as pd

import os
import sys

import nibabel as nib 
import imageio
from PIL import Image

In [ ]:
# transform all of Train's 3D data into  2D images
np.set_printoptions(threshold=np.inf)
def nii_to_image(niifile):
  return 0
dirname = "./Train"
# determine if the folder dirname exists
if not os.path.exists(dirname):
    print("error: folder \"", dirname, "\" not exits!")
    sys.exit()

        
# get recursively all the filenames under folders and subfolders
for root,dirs,files in os.walk(dirname):
    for file in files:
        img_path = os.path.join(root,file)
        # print(img_path)
        img = nib.load(img_path) #load .nii
        img_fdata = img.get_fdata()
        fname = file.replace('.nii','')
        s = file[-3:]
        if s != 'nii':
          continue
        s1 = file[:-4]
        print(s1)

# start transforming to 2D image
        (x,y,z) = img.shape
        print("img:")
        print(img.shape) # output the size of each data

        result_path = os.path.join('./Train_img',fname)
        if not os.path.exists(result_path):
            os.mkdir(result_path)

        for i in range(z): #split by z-axis to obtain the axial plane
          silce = img_fdata[:, :, i] 
          silce = (silce*255.0).astype('uint8')
          imageio.imwrite(os.path.join(result_path,'{}.png'.format(i)), silce)
          img = Image.open(os.path.join(result_path,'{}.png'.format(i)))
          img.save(os.path.join(result_path,'{}.png'.format(i)))


In [ ]:
# transform all of Test's 3D data into  2D images
np.set_printoptions(threshold=np.inf)
def nii_to_image(niifile):
  return 0
dirname = "./Test"
# determine if the folder dirname exists
if not os.path.exists(dirname):
    print("error: folder \"", dirname, "\" not exits!")
    sys.exit()

        
# get recursively all the filenames under folders and subfolders
for root,dirs,files in os.walk(dirname):
    for file in files:
        img_path = os.path.join(root,file)
        print(img_path)
        img = nib.load(img_path) #load .nii
        img_fdata = img.get_fdata()
        fname = file.replace('.nii','')
        s = file[-3:]
        if s != 'nii':
          continue
        s1 = file[:-4]
        print(s1)

# start transforming to 2D image 
        (x,y,z) = img.shape
        print("img:")
        print(img.shape)

        result_path = os.path.join('./Test_img',fname)
        if not os.path.exists(result_path):
            os.mkdir(result_path)

        for i in range(z): #split by z-axis to obtain the axial plane
          silce = img_fdata[:, :, i] 
          silce = (silce*255.0).astype('uint8')

          imageio.imwrite(os.path.join(result_path,'{}.png'.format(i)), silce)
          img = Image.open(os.path.join(result_path,'{}.png'.format(i)))
          img.save(os.path.join(result_path,'{}.png'.format(i)))


In [ ]:
# find the mask of the left atrium and write it to the folder mask_LV as a binary mask, 
#  which will be used as the label for the next segmentation step
dirname = "./Train_img"

for root,dirs,files in os.walk(dirname):
    for file in files:
        s = root[-3:]
        if s == 'seg': #find the folder with the labeled data
          img_path = os.path.join(root,file)
          print(img_path)
          img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
          for i in range(img.shape[0]):
            for j in range(img.shape[1]):
              # 253 corresponds to the left ventricle
              if img[i][j] == 253 : 
                img[i][j] = 255
              else:
                img[i][j] = 0
          result_path = os.path.join('./Mask_LV',root[-10:])
          if not os.path.exists(result_path):
              os.mkdir(result_path)
          imageio.imwrite(os.path.join(result_path,file), img) #write to the mask_lv/00x_Ex_seg folder 
          mask = Image.open(os.path.join(result_path,file))
          mask.save(os.path.join(result_path,file))
        else:
          continue

  


In [ ]:
# find the position of the left ventricle in the labeled 2D data, set it to 0, and leave the rest as it is
# This will be used as input in the segmentation section (2D image with only the right ventricle and myocardial part labeled)
dirname = "./Train_img"

for root,dirs,files in os.walk(dirname):
    for file in files:
        s = root[-3:]
        if s == 'seg': #find the folder with the labeled data
          img_path = os.path.join(root,file)
          print(img_path)
          img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
          for i in range(img.shape[0]):
            for j in range(img.shape[1]):
              # 244 corresponds to the myocardial part, 255 corresponds to the right ventricle
              if img[i][j] == 253 : 
                img[i][j] = 0
          result_path = os.path.join('./Mask_noLV',root[-10:])
          if not os.path.exists(result_path):
              os.mkdir(result_path)
          imageio.imwrite(os.path.join(result_path,file), img) #write to the mask_nolv/00x_Ex_seg folder
          mask = Image.open(os.path.join(result_path,file))
          mask.save(os.path.join(result_path,file))
        else:
          continue